In [41]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from bs4 import BeautifulSoup, element
import time as tm
import os
import requests

In [52]:
chars_to_remove = ",^\()|\"\n"

In [42]:
# this function constructs the url for an archived webpage according to https://wayback.archive-it.org formatting
def get_html(collection_id, folder, data):

    # creates a local directory named 'folder' if it doesn't exist
    if not os.path.isdir(folder):
        os.mkdir(folder)

    # the base url for the archive
    ai_url = "https://wayback.archive-it.org"

    # this gets a unique list of dates and urls so we can avoid duplication
    date_list = pd.to_datetime(data['date'].unique())
    urls = data['url'].unique()

    # this loop iterates through all the urls, searches for the archived webpage
    # then looks for any hyperlinks on the page that are associated with a date in the list
    for url in urls:

        # this is the url of an archived webpage's home 
        # (e.g. https://wayback.archive-it.org/12706/*/http://mypaperonline.com)
        home = ai_url + '/' + str(collection_id) + '/*/' + url

        # this block gets all the date hyperlinks
        page = requests.get(home)
        soup = BeautifulSoup(page.content, "lxml")
        alist = soup.find_all("a")
        alist = [a for a in alist if 'onclick' in a.attrs]
        dates = [dt.strptime(a.text, '%b %d, %Y') for a in alist]

        # this loops through all the date hyperlinks and if they're in
        # the date_list it saves them locally
        for a in alist:

            # this creates a local path for the html, it converts URLS+dates
            # into unique simple strings
            path = folder + '/' + dt.strptime(a.text, '%b %d, %Y').strftime('%Y%m%d') + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', url.split('/')[-1])) + '.html'

            # this is where it saves the html ('onclick' in a.attrs is probably redundant)
            if 'onclick' in a.attrs and dt.strptime(a.text, '%b %d, %Y') in date_list and not os.path.isfile(path):
                wp = requests.get('https:' + a.attrs['href'])
                f = open(path, 'wb')
                f.write(wp.content)
                f.close()

In [43]:
chosen = pd.read_csv('final_chosen.csv')

In [44]:
get_html(12706, 'chosenSamp', chosen)

In [49]:
ctr = 0
folder = 'chosenSamp'
chosen['article'] = 0

for index, row in chosen.iterrows():

    path = folder + '/' + ''.join(re.findall('\d+', row.date)) + '_' + ''.join(re.findall('(\d+|[a-zA-Z]+|-|\.)', row.url.split('/')[-1])) + '.html'

    f = open(path, 'rb')
    soup = BeautifulSoup(f.read(), "lxml")
    articles = soup.find_all("article")

    if len(articles) == 1: 
        ctr = ctr + 1
        text = articles[0].text.translate ({ord(c): "" for c in chars_to_remove})
        chosen.loc[index, 'text'] = text
        print(path)
        print(text)
        chosen.loc[index, 'article'] = 1
    else:
        chosen.loc[index, 'text'] = 'no article'
        chosen.loc[index, 'article'] = 0

    f.close()

    tm.sleep(.1)


chosenSamp/20191001_.html


Home  Entertainment  The Fanfare and National Scope to Support the Challenges of Bennie Boom’s...
EntertainmentMoviesNewsOpinion/Blog

The Fanfare and National Scope to Support the Challenges of Bennie Boom’s Tupac Movie

By NewarkBlack -  June 9, 2017 


SHARE

Facebook
Twitter


tweet 


By Maurice Brian Henderson
Not often has there been a clarion call, a harkening and lyceum for people, viewers and patrons to proceed with action.  Bennie Boom’s soon to be released movie on Tupac Shakur entitled “All Eyez on Me” is surely the suggest to do so.

The value proposition is that it is important to support this film as the opus reconsideration of Black males as Endangered Species and America Most Wanted.  I know that this is the period of enlightenment which warrants a film like this as a social paradigm shifting. It is the circumstance of work left undone.  Bennie knows well about the male experience, he has long since joined the ranks of Alpha Phi Alpha Frate

In [50]:
print(ctr)

554


In [51]:
chosen.drop('Unnamed: 0', axis=1).to_csv('chosen2.csv')